In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import scipy
import os
import pickle
import sys
import librosa
import librosa.display
from IPython.display import Audio
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, normalize
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers, models
import IPython
import math
import json
from tensorflow.keras import layers, models
!pip install ffmpeg gstreamer

# PREDICTING FROM FEATURES GIVEN

Referenced from blog.clairvoyantsoft.com/music-genre-classification-using-cnn-ef9461553726


In [ ]:
os.chdir('/content/gdrive/MyDrive/GTZAN')

In [ ]:
df = pd.read_csv('Data/features_3_sec.csv')
df.head()

FEATURE EXTRACTION

In [ ]:
#encode y-labels
class_list = df.iloc[:,-1]
converter = LabelEncoder()
y=converter.fit_transform(class_list)

In [ ]:
df =df.drop(labels='filename',axis=1)
#to normalize all the x data
from sklearn.preprocessing import StandardScaler
fit = StandardScaler()
X = fit.fit_transform(np.array(df.iloc[:,:-1],dtype=float))

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)
print(len(y_test),len(y_train))

In [ ]:
X_train = X_train.reshape(X_train.shape[0],X_train.shape[1],1)
input_shape = X_train.shape
X_train.shape

best val_acc : 91.44%

ANN SOLUTION

In [ ]:
#ANN for features given
model = models.Sequential()
model.add(layers.Dense(512, activation='relu',input_shape=(input_shape[1],), name='dense1'))
model.add(layers.Dropout(0.2))
model.add(layers.Dense(256, activation='relu',name='dense2'))
model.add(layers.Dropout(0.2))
model.add(layers.Dense(128, activation='relu',name='dense3'))
model.add(layers.Dropout(0.2))
model.add(layers.Dense(64, activation='relu',name='dense4'))
model.add(layers.Dropout(0.2))
model.add(layers.Dense(10, activation='softmax'))

model.compile(loss="sparse_categorical_crossentropy",optimizer="adam",metrics=["accuracy"])
model.summary()
model.fit(X_train.squeeze(axis=2),y_train, validation_data=(X_test, y_test), epochs = 100, batch_size=64)


# USING AUDIO SIGNALS
Referenced from:

https://www.youtube.com/watch?v=szyGiObZymo

https://www.youtube.com/watch?v=dOG-HxpbMSw

Changed the CNN model to use Conv1D instead of 2D, test accuracy improved compared to video by 3%.

In [ ]:
os.chdir('/content/gdrive/MyDrive/GTZAN/Data/genres_original')
os.getcwd()

In [ ]:
track_dur = 30
sample_rate = 22050 ## default, change in librosa.load if changing here and vice versa
samples_per_track = sample_rate * track_dur
def save_mfcc(n_mfcc= 13, n_fft= 2048,hop_length=512, num_segments=5):
  num_samples_per_segment = int(samples_per_track/num_segments)
  mfcc_per_segment = math.ceil(num_samples_per_segment / hop_length)
  data = {
      "mapping":[],
      "mfcc":[],
      "labels":[]
  }
  for i, (dir,_,filenames) in enumerate(os.walk(os.getcwd())):
    if dir == os.getcwd():
      data["mapping"]=filenames
    else:
      label = dir.split('/')[-1]
      for f in filenames:
        # print('file',f)
        file_path = os.path.join(dir,f)
        try:
          signal,sr = librosa.load(file_path)##can add sample rate here
          for s in range(num_segments):
            start = s * num_samples_per_segment
            end = start + num_samples_per_segment
            mfcc = librosa.feature.mfcc(signal[start:end],sr,n_fft=n_fft,n_mfcc=n_mfcc,hop_length=hop_length).T
            #ensure same training sample size
            if(len(mfcc)== mfcc_per_segment):
              data["mfcc"].append(mfcc)
              data["labels"].append(i-1)
        except:
          print('skipping',f)
          continue
  return data
    # json.dump(data,open('/content/gdrive/MyDrive/GTZAN/Data/audio_data.json','w'),indent=4)

    

In [ ]:
# data = save_mfcc(num_segments=10)
# data_json = data.copy()
# data_json['mfcc'] = [a.tolist() for a in data_json['mfcc']]
# json.dump(data_json,open('/content/gdrive/MyDrive/GTZAN/Data/audio_data.json','w'),indent=4)
data = json.load(open('/content/gdrive/MyDrive/GTZAN/Data/audio_data.json','r'))


In [ ]:
X, y = np.array(data["mfcc"]),np.array(data["labels"])
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.25)
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size = 0.2)
# X_train = X_train.reshape((X_train.shape[0],X_train.shape[1],1))
# X_val = X_val.reshape((X_val.shape[0],X_val.shape[1],1))
# X_test = X_test.reshape((X_test.shape[0],X_test.shape[1],1))

input_shape = (X_train).shape


In [ ]:
from tensorflow.keras import layers, models
model = models.Sequential()
model.add(layers.Conv1D(512, kernel_size=10, activation='relu', input_shape=input_shape[1:],name='conv1'))
model.add(layers.MaxPooling1D(2))
model.add(layers.Conv1D(512, kernel_size=5, activation='relu',name='conv2'))
model.add(layers.MaxPooling1D(2))
model.add(layers.Flatten())
model.add(layers.Dense(256, activation='relu',name='conv3'))
model.add(layers.Dropout(0.2))
model.add(layers.Dense(128, activation='relu'))
model.add(layers.Dropout(0.2))
model.add(layers.Dense(10, activation='softmax'))

model.compile(loss="sparse_categorical_crossentropy",optimizer="adam",metrics=["accuracy"])
model.summary()

filepath = 'audio_model.epoch{epoch:02d}-loss{val_loss:.2f}.hdf5'
checkpoint = ModelCheckpoint(filepath=filepath, monitor=’val_accuracy’, verbose=1, save_best_only=True, mode="max")
model.fit(X_train,y_train, validation_data=(X_val, y_val), callbacks = [checkpoint], epochs = 50, batch_size=64)


In [ ]:
model.evaluate(X_test, y_test, verbose = 1)

In [ ]:
# ##best model save
# model = models.Sequential()
# model.add(layers.Conv1D(512, kernel_size=10, activation='relu', input_shape=input_shape[1:],name='conv1'))
# model.add(layers.MaxPooling1D(2))
# model.add(layers.Conv1D(512, kernel_size=5, activation='relu',name='conv2'))
# model.add(layers.MaxPooling1D(2))
# model.add(layers.Flatten())
# model.add(layers.Dense(256, activation='relu',name='conv3'))
# model.add(layers.Dropout(0.2))
# model.add(layers.Dense(128, activation='relu'))
# model.add(layers.Dropout(0.2))
# model.add(layers.Dense(10, activation='softmax'))

# model.compile(loss="sparse_categorical_crossentropy",optimizer="adam",metrics=["accuracy"])
# model.summary()
# model.fit(X_train,y_train, validation_data=(X_test, y_test), epochs = 60, batch_size=64)
